In [2]:
import os
import sys
import time
import glob
import fitsio

import itertools
import warnings

import numpy as np
import pylab as pl

import desisim.templates
import astropy.io.fits              as      fits

import desispec.io
import redrock.templates
import matplotlib.pyplot            as      plt

from   os                           import  path
from   astropy.convolution          import  convolve, Box1DKernel
from   desispec.spectra             import  Spectra
from   desispec.frame               import  Frame
from   desispec.resolution          import  Resolution
from   desispec.io.meta             import  findfile
from   desispec.io                  import  read_frame, read_fiberflat, read_flux_calibration, read_sky, read_fibermap
from   desispec.interpolation       import  resample_flux
from   astropy.table                import  Table
from   desispec.io.image            import  read_image
from   specter.psf.gausshermite     import  GaussHermitePSF
from   scipy.signal                 import  medfilt
from   desispec.calibfinder         import  CalibFinder
from   astropy.utils.exceptions     import  AstropyWarning
from   scipy                        import  stats
from   pathlib                      import  Path
from   templateSNR                  import  templateSNR
from   RadLSS                       import  RadLSS

ModuleNotFoundError: No module named 'templateSNR'

In [5]:
! cd /global/project/projectdirs/desi/users/mjwilson/repos/desispec/py/desispec/radlss

In [6]:
! ls

BootCalib_CRs.ipynb	     Overscan.ipynb
Boot_Calibs_firsttry.ipynb   Preproc.ipynb
Boot_Calibs_fulltest.ipynb   QA_Brick.ipynb
Bootcalib_r.ipynb	     QA_Exposure.ipynb
Bootcalib_z.ipynb	     QA_Frame.ipynb
Bootstrap_tests.ipynb	     QA_Prod.ipynb
Cosmics.ipynb		     QA_S2N_Develop.ipynb
Holy_grail.ipynb	     SpectroPerfectCoadds.ipynb
Intro_to_DESI_spectra.ipynb  TemplateSNR.ipynb
JoinCosmics.ipynb	     cosmic-joiner.ipynb


In [ ]:
# warnings.simplefilter('ignore', category=AstropyWarning)

# Foundations to desispec ticket 1004:  Template SNR and LSS rad. weights.

Template SNR:

https://desi.lbl.gov/DocDB/cgi-bin/private/RetrieveFile?docid=4723;filename=sky-monitor-mc-study-v1.pdf;version=2

applied to SV0 dataset:

https://desi.lbl.gov/trac/wiki/TargetSelectionWG/SV0

# Template SNR

$\Huge \langle tSNR^2 \rangle = \sum_{grz} \sum_{\lambda \in grz} \left ( \frac{\tau^2 T^2 \langle \Delta F^2 \rangle}{\sigma_e^2 \cdot {\rm NPIX/A} + \tau T_{\rm sky}  F_{\rm sky}} \right ) $

_"We choose to normalize it such that zSNR = 3 for a 1000 sec dark time exposure in
nominal conditions."_

Here:  
- $\sigma_e$ is the READNOISE in units of ELECTRONS / PIXEL. 
- NPIX/A is the projected fiber area in units of PIXELS for a width of **ONE ANGSTROM**.  
- $\Delta F$ is the residual (template) flux from a 100A filtered equivalent.
- $\tau$ is the exposure time, already included in the flux calib.
- $T_{\rm sky} F_{\rm sky}$ is the estimated fiber sky flux in units of ELECTRONS PER ANGSTROM.

In [11]:
def get_expids(night, andes='/global/cfs/cdirs/desi/spectro/redux/andes'):
    tiles  = np.unique(np.array([x.split('/')[-3] for x in glob.glob(andes + '/tiles/*/{}/cframe-*'.format(night))]).astype(np.int))

    # np.sort(np.array([x.split('/')[-1] for x in glob.glob(andes + '/exposures/{}/*'.format(night))]).astype(np.int))                                                                                  
    expids = np.unique(np.array([x.split('/')[-1].split('-')[2].replace('.fits','') for x in glob.glob(andes + '/tiles/*/{}/cframe-*'.format(night))]).astype(np.int))

    return  expids, tiles

In [12]:
nmax     = 1
night    = '20200315'
tracers  = ['ELG']

expids, tiles = get_expids(night)

camera   = ['b5', 'r5', 'z5']

In [13]:
expids

array([55587, 55589, 55590, 55591, 55592, 55593, 55594, 55611, 55612,
       55613, 55626, 55627, 55628, 55639, 55640, 55641, 55642, 55643,
       55654, 55655, 55656, 55668, 55670, 55672, 55674, 55676, 55678,
       55680, 55682, 55684, 55686, 55688, 55690, 55692])

In [1]:
for nexp, expid in enumerate(expids[2:]):
    print('Solving for EXPID {:08d}'.format(expid))
    
    rads = RadLSS(night, expid, cameras=cameras)
    
    rads.compute(tracers=tracers)

    if nexp == nmax:
        break

NameError: name 'expids' is not defined

# Done.